# IMDB Data Analysis

Shenyue Jia

[jiashenyue.info](https://jiashenyue.info)

In [34]:
# data wrangling
import numpy as np
import pandas as pd
import os

# database
import pymysql
from sqlalchemy import create_engine
from urllib.parse import quote_plus as urlquote

# api
import json

## Download data

### `basics`

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [3]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


## `ratings`

In [4]:
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [5]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1957
1,tt0000002,5.8,263
2,tt0000003,6.5,1791
3,tt0000004,5.6,179
4,tt0000005,6.2,2594


## `akas`

In [6]:
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [7]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


### Save original data

In [35]:
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['title_akas_2000_2021.csv.gz',
 'title_basics_2000_2021.csv.gz',
 'title_ratings_2000_2021.csv.gz']

In [36]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics_large.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas_large.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings_large.csv.gz",compression='gzip',index=False)

## Prepare data

### Clean and filter `basic` data

- Replace null values in the data from `\` to null

In [8]:
df_basics = basics.replace({'\\N':np.nan})

In [9]:
df_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9661361 entries, 0 to 9661360
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 663.4+ MB


- Eliminate movies that are null for `runtimeMinutes`

In [10]:
df_basics = df_basics[pd.notnull(df_basics['runtimeMinutes'])]

- Eliminate movies that are null for `genres`

In [11]:
df_basics = df_basics[pd.notnull(df_basics['genres'])]

- Keep only `titleType`==`Movie`

In [12]:
df_basics = df_basics.query("titleType == 'movie'")

- Keep `startYear` 2000-2021
  - `startYear` is a string field

In [13]:
years = list(range(2000,2022,1))
years_str = list(map(str, years))
years_str

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021']

In [14]:
# filter data with startYear among the values in years_str
df_basics = df_basics[df_basics['startYear'].isin(years_str)]
df_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,133,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"


- Eliminate movies that include "Documentary" in `genre`

In [15]:
 # Exclude movies that are included in the documentary category.
is_documentary = df_basics['genres'].str.contains('documentary',case=False)
df_basics = df_basics[~is_documentary]

In [16]:
# check if there are still rows with generes column contain documentary
df_basics[df_basics['genres'].str.contains('documentary')]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


- Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [17]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =df_basics['tconst'].isin(akas['titleId'])
keepers

34803      True
61116      True
67669      True
77964      True
86801      True
           ... 
9661034    True
9661043    True
9661082    True
9661127    True
9661211    True
Name: tconst, Length: 137980, dtype: bool

In [18]:
df_basics = df_basics[keepers]
df_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


- Check the dimension of `df_basics` after cleaning and filtering

In [19]:
df_basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137367 entries, 34803 to 9661211
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          137367 non-null  object
 1   titleType       137367 non-null  object
 2   primaryTitle    137367 non-null  object
 3   originalTitle   137367 non-null  object
 4   isAdult         137367 non-null  object
 5   startYear       137367 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  137367 non-null  object
 8   genres          137367 non-null  object
dtypes: object(9)
memory usage: 10.5+ MB


### Clean and filter `akas` data

- Keep only US movies

In [20]:
# filter by region
df_akas = akas.query("region == 'US'")
df_akas.shape

(1418920, 8)

- Replace null values in the data from `\` to null

In [21]:
df_akas = df_akas.replace({'\\N':np.nan})
df_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


- Check the dimension of `df_akas` after cleaning and filtering

In [22]:
df_akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1418920 entries, 5 to 35139161
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1418920 non-null  object
 1   ordering         1418920 non-null  int64 
 2   title            1418920 non-null  object
 3   region           1418920 non-null  object
 4   language         3843 non-null     object
 5   types            974876 non-null   object
 6   attributes       46118 non-null    object
 7   isOriginalTitle  1417575 non-null  object
dtypes: int64(1), object(7)
memory usage: 97.4+ MB


### Clean and filter `ratings` data

In [23]:
df_ratings = ratings.replace({'\\N':np.nan})
df_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1957
1,tt0000002,5.8,263
2,tt0000003,6.5,1791
3,tt0000004,5.6,179
4,tt0000005,6.2,2594


- Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [24]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =df_ratings['tconst'].isin(akas['titleId'])
keepers

0           True
1           True
2           True
3           True
4           True
           ...  
1285275     True
1285276     True
1285277    False
1285278    False
1285279    False
Name: tconst, Length: 1285280, dtype: bool

In [25]:
df_ratings = df_ratings[keepers]
df_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1957
1,tt0000002,5.8,263
2,tt0000003,6.5,1791
3,tt0000004,5.6,179
4,tt0000005,6.2,2594


- Check the dimension of `df_ratings` after cleaning and filtering

In [26]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 963728 entries, 0 to 1285276
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         963728 non-null  object 
 1   averageRating  963728 non-null  float64
 2   numVotes       963728 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 29.4+ MB


## Saving files

In [37]:
## Save current dataframe to file.
df_basics.to_csv("Data/title_basics_filtered.csv.gz",compression='gzip',index=False)
df_akas.to_csv("Data/title_akas_filtered.csv.gz",compression='gzip',index=False)
df_ratings.to_csv("Data/title_ratings_filtered.csv.gz",compression='gzip',index=False)

In [38]:
# test if files are successfully saved
df_basics = pd.read_csv("Data/title_basics_filtered.csv.gz", low_memory = False)
df_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
